# Estimator Benchmarks

Notebook for benchmarking known IV estimators against different data generating processes.

Current roster:
- Split-sample IV
- 2SLS
- Jackknife IV
- LIML **TODO**
- Mostly harmless ML **TODO**
- DeepIV **TODO**
- DoubleML **TODO**

# Imports

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from linearmodels.iv import IV2SLS

In [9]:
# this is assuming a notebook placed inside the notebooks/ folder
import sys
sys.path.append('../src/data')
from lin_norm_generator import LinearNormalDataGenerator

df = LinearNormalDataGenerator().generate()

# TabPFN-style data generating process

In [10]:
import torch
from torch import nn
from torch.distributions import MultivariateNormal
import torch.nn.functional as F

In [43]:
MAX_VARS = 100

class GaussianNoise(nn.Module):
    """
    Lifted from https://github.com/automl/TabPFN/blob/main/tabpfn/priors/mlp.py
    TODO introduce shared covariance matrix
    """
    def __init__(self, std, device):
        super().__init__()
        self.std = std
        self.device=device

    def forward(self, x):
        return x + torch.normal(torch.zeros_like(x), self.std)


def causes_sampler_f(num_causes):
    means = np.random.normal(0, 1, (num_causes))
    std = np.abs(np.random.normal(0, 1, (num_causes)) * means)
    return means, std

class IVGenerator(nn.Module):
    """
    Neural network to generate IV datasets.

    TODO add ability to generate IV controls to cover the Angrist/Frandsen case
    """
    def __init__(self, 
                tau: float,
                max_vars: int,
                n_instruments: int,
                instrument_covariance: torch.Tensor,
                instrument_strength: float, # this currently translates to mu^2/n_samples
                tau_activation: str = 'identity',
                instrument_activation: str = 'identity',
                # TODO add controls as well
                control_activation: str = 'identity',
                control_covariance: torch.Tensor = None, 
                control_str: float = 0,
                n_controls: int = 0,

                ):
        super().__init__()

        # TODO are these actually needed?
        #self.controls = nn.Linear(max_vars, 1)
        self.instruments = nn.Linear(max_vars, 1)

        #Nself.n_controls = n_controls
        self.n_instruments = n_instruments

        if control_covariance is not None:
            assert control_covariance.shape == (n_controls, n_controls)

        assert instrument_covariance.shape == (n_instruments, n_instruments)
        self.instrument_covariance = instrument_covariance
        self.instrument_sampler = MultivariateNormal(torch.zeros(self.n_instruments), self.instrument_covariance)
        
        # currently follows the Beta pattern of Lennon et al. 2022, corresponds to their pi
        self.instrument_coefficients = torch.pow(torch.ones(self.n_instruments) * 0.7, 
                                         torch.arange(0, self.n_instruments))

        # shuffle coefficients
        self.instrument_coefficients = self.instrument_coefficients[torch.randperm(self.n_instruments)]

        self.sigma_v = (torch.t(self.instrument_coefficients) @ self.instrument_covariance @ self.instrument_coefficients) / instrument_strength
        self.sigma_v = torch.sqrt(self.sigma_v)
        self.sigma_y = 1
        
        # TODO need to ensure that confounder covariance is positive definite
        self.confound_covariance = torch.Tensor([[self.sigma_y**2, self.sigma_y*self.sigma_v], 
                                                 [self.sigma_y*self.sigma_v, self.sigma_v**2]])
        self.confound_sampler = MultivariateNormal(torch.zeros(2), self.confound_covariance)

        self.tau = tau
        self.control_str = control_str
        
        self.activations = {
            'identity': lambda x: x,
            'relu': F.relu,
            'sigmoid': F.sigmoid,
            'tanh': F.tanh,
            'softplus': F.softplus,
            'leaky_relu': F.leaky_relu,
            'elu': F.elu,
        }
    
        self.tau_activation = self.activations[tau_activation]
        self.instrument_activation = self.activations[instrument_activation]
        #self.confounder_activation = self.activations[confounder_activation]


    def forward(self):
        """Generates a single data sample"""
        #confound_sample = torch.cat([torch.randn(self.n_confounds), torch.zeros(MAX_VARS - self.n_confounds)])

        # noise sample [\epislon_y, \epeilson_v] according to confounder covariance
        #noise_sample = torch.normal(TODO)
        instrument_sample = torch.cat([self.instrument_sampler.sample(), torch.zeros(MAX_VARS - self.n_instruments)])

        epislon_y, epsilon_v = self.confound_sampler.sample()
        #confounds = self.confounder_activation(self.confounders(confound_sample))
        pi = torch.cat([self.instrument_coefficients, torch.zeros(MAX_VARS - self.n_instruments)])
        treat = self.instrument_activation(torch.t(pi) @ instrument_sample) + epsilon_v
        
        #print(treat.shape)

        outcome = self.tau*self.tau_activation(treat) + torch.randn(1) + epislon_y

        # return data matrix of T, Y, Z
        return torch.cat([torch.Tensor([treat, outcome]), instrument_sample])

        # return data matrix of T, Y, X, Z
        #return torch.cat([treat, outcome, confound_sample, instrument_sample])
        
    def batch(self, batch_size: int):
        """Generate batch of examples"""
        return torch.stack([self.forward() for _ in range(batch_size)])


In [44]:
n_samples = 100
mu2 = 2000
iv_gen = IVGenerator(
            tau=1,
            max_vars=MAX_VARS,
            n_instruments=7,
            instrument_covariance=torch.Tensor([[1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                                                [0.5, 1, 0.5, 0.5, 0.5, 0.5, 0.5],
                                                [0.5, 0.5, 1, 0.5, 0.5, 0.5, 0.5],
                                                [0.5, 0.5, 0.5, 1, 0.5, 0.5, 0.5],
                                                [0.5, 0.5, 0.5, 0.5, 1, 0.5, 0.5],
                                                [0.5, 0.5, 0.5, 0.5, 0.5, 1, 0.5],
                                                [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1]]), 
            instrument_strength=mu2/n_samples,
        )

data = iv_gen.batch(n_samples).detach().numpy()

In [45]:
data.shape

(100, 102)

# Lennon et al. 2022 Figure 2 Replication

TODO